![logo](https://user-images.githubusercontent.com/59526258/124226124-27125b80-db3b-11eb-8ba1-488d88018ebb.png)
> **Copyright (c) 2021 CertifAI Sdn. Bhd.**<br>
 <br>
This program is part of OSRFramework. You can redistribute it and/or modify
<br>it under the terms of the GNU Affero General Public License as published by
<br>the Free Software Foundation, either version 3 of the License, or
<br>(at your option) any later version.
<br>
<br>This program is distributed in the hope that it will be useful,
<br>but WITHOUT ANY WARRANTY; without even the implied warranty of
<br>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
<br>GNU Affero General Public License for more details.
<br>
<br>You should have received a copy of the GNU Affero General Public License
<br>along with this program.  If not, see <http://www.gnu.org/licenses/>.
<br>

# CRUD Operations Using `psycopg2`

## Introduction 
This tutorial is guide readers on how to use `psycopg2` library<sup>[[1]](#1)</sup> to perform create, read, update and delete (CRUD) operations.

## What will we accomplish?
By using `psycopy2`, readers are able to perform the following in Python:
1. Create tables and rows in `PostgreSQL`,
2. Read results of tables and rows in `PostgreSQL`,
3. Update tables and rows in `PostgreSQL`, and
4. Delete tables and rows in `PostgreSQL`.

## Instruction
You can follow along this notebook by yourself or alongside the instructor.

## Notebook Content
* [Create Operations](#create)
* [Read Operations](#read)
* [Update Operations](#update)
* [Delete Operations](#delete)
* [Exercise](#exercise)
* [References](#references)

## <a name="create">Create Operations</a>
Create operations are done using `INSERT` SQL commands. These commands are used to insert new data into an existing database.

Before we get into the actual operations, let's do some setup work first that also serves as a recap of previous notebook - `Introduction to psycopg2`.

### Recap

In [28]:
# Import package
import psycopg2
from psycopg2 import sql

# Establish connection to Postgres server
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="asd",
    database="postgres"
)

# Create a new database
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("CREATE DATABASE psycopg;")

# Disconnect from "postgres" DB
cursor.close()
conn.close()

# Establish connection to newly created "psycopy" DB
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="asd",
    database="psycopg"
)

# Create cursor object
conn.autocommit = True
cursor = conn.cursor()

Let's create a new table and populate it with some data.

In [29]:
query = """CREATE TABLE country
(
    country_id INTEGER,
    country_name VARCHAR(50) NOT NULL,
    country_code VARCHAR(2) UNIQUE NOT NULL,
    population INTEGER,
    yearly_change NUMERIC(5, 2),
    PRIMARY KEY (country_id)
);
"""
cursor.execute(query)


In [30]:
# First way of INSERT query by using parameters
query = """
    INSERT INTO country (country_id, country_name, country_code, population, yearly_change)
    VALUES (%s, %s, %s, %s, %s);
"""
cursor.execute(query, (1, 'Malaysia', 'MY', 32365999, 1.30,))

In [31]:
# Alternative way of INSERT query
query = """
    INSERT INTO country (country_id, country_name, country_code, population, yearly_change)
VALUES
    (2, 'Singapore', 'SG', 5850342, 0.79),
    (3, 'Indonesia', 'ID', 126476461, -0.30);
"""
cursor.execute(query)

We can validate that the table is created by querying the `pg_catalog.pg_tables` catalog. `WHERE` condition is used to filter out irrelevant system tables.

In [32]:
query = """
    SELECT pg_tables.tablename
    FROM pg_catalog.pg_tables
    WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';
"""
cursor.execute(query)
cursor.fetchall()


[('country',)]

##  <a name="read">Read Operations</a>

Read operations pertain with reading the inserted data. This operation corresponds with `SELECT` command in SQL. Below are some demonstrations on how to read data from tables in database.

The resultant `conn` is now a `connection` object which encapsulates a database session.

In [33]:
SELECT_ALL_QUERY = "SELECT * FROM {};".format('country')
cursor.execute(sql.SQL(SELECT_ALL_QUERY))
# Show what is the actual query executed in Postgresql
print(cursor.query)

cursor.fetchall()

b'SELECT * FROM country;'


[(1, 'Malaysia', 'MY', 32365999, Decimal('1.30')),
 (2, 'Singapore', 'SG', 5850342, Decimal('0.79')),
 (3, 'Indonesia', 'ID', 126476461, Decimal('-0.30'))]

You might wonder what each column means. We can call `description` attribute for the cursor to display column name.

In [34]:
cursor.description

(Column(name='country_id', type_code=23),
 Column(name='country_name', type_code=1043),
 Column(name='country_code', type_code=1043),
 Column(name='population', type_code=23),
 Column(name='yearly_change', type_code=1700))

Let's just display the columns `country_name` and `yearly_change` only.

In [35]:
query = """
    SELECT {},{} FROM {};
    """.format('country_name', 'population', 'country')
cursor.execute(sql.SQL(query))
print(cursor.description)
cursor.fetchall()


(Column(name='country_name', type_code=1043), Column(name='population', type_code=23))


[('Malaysia', 32365999), ('Singapore', 5850342), ('Indonesia', 126476461)]

##  <a name="update">Update Operations</a>
Update operations pertain with the act of modifying existing data. The corresponding SQL command for this type of operation is, wait for it, `UPDATE` command.

In [36]:
update_query = """
    UPDATE {}
    SET population = {}, yearly_change = {}
    WHERE country_id = 3
""".format('country', 9999, 1.07)
cursor.execute(sql.SQL(update_query))
cursor.execute(SELECT_ALL_QUERY)
cursor.fetchall()

[(1, 'Malaysia', 'MY', 32365999, Decimal('1.30')),
 (2, 'Singapore', 'SG', 5850342, Decimal('0.79')),
 (3, 'Indonesia', 'ID', 9999, Decimal('1.07'))]

## <a name="delete">Delete Operations</a>

Last but not least, delete operations are used to remove existing data. The corresponding SQL command is `DELETE`.

In [37]:
query = """
    DELETE FROM {}
    WHERE country_name = '{}'
""".format('country', 'Malaysia')
cursor.execute(sql.SQL(query))

We can also remove the table from the database.

In [38]:
cursor.execute(sql.SQL("DROP TABLE IF EXISTS {};".format('country')))

## <a name="exercise">Exercise</a>

Below are exercises to practice what we have just learnt. Follow along the instructions provided in the comment.

In [39]:
# Create titanic_train table by following the schema provided 
#    at https://www.kaggle.com/c/titanic/data?select=train.csv
query = """
CREATE TABLE titanic_train
(
    PassengerId INTEGER PRIMARY KEY,
    Survived BOOLEAN NOT NULL,
    Pclass INTEGER NOT NULL,
    Name VARCHAR(255) NOT NULL,
    Sex CHAR(10) NOT NULL,
    Age REAL,
    SibSp INTEGER NOT NULL,
    Parch INTEGER NOT NULL,
    Ticket VARCHAR(255) NOT NULL,
    Fare REAL NOT NULL,
    Cabin CHAR(20),
    Embarked CHAR(3)
);
"""
cursor.execute(query)

# Inspect created table schema
query = """
    SELECT column_name, data_type, character_maximum_length, is_nullable
    FROM INFORMATION_SCHEMA.COLUMNS 
    WHERE table_name = 'titanic_train';"""

cursor.execute(query)
cursor.fetchall()


[('passengerid', 'integer', None, 'NO'),
 ('survived', 'boolean', None, 'NO'),
 ('pclass', 'integer', None, 'NO'),
 ('age', 'real', None, 'YES'),
 ('sibsp', 'integer', None, 'NO'),
 ('parch', 'integer', None, 'NO'),
 ('fare', 'real', None, 'NO'),
 ('cabin', 'character', 20, 'YES'),
 ('name', 'character varying', 255, 'NO'),
 ('sex', 'character', 10, 'NO'),
 ('ticket', 'character varying', 255, 'NO'),
 ('embarked', 'character', 3, 'YES')]

In [40]:
# Get valid dataset path
from pathlib import Path
dataset_path = str(Path().resolve().parent/"data/titanic_train.csv")

# Copy over data from csv file to DB
query = """
COPY titanic_train
FROM '{0}'
DELIMITER ','
CSV HEADER;
""".format(dataset_path)
cursor.execute(query)

In [41]:
# Read first 3 rows ordered by PassengerId
cursor.execute("SELECT * FROM titanic_train ORDER BY PassengerId LIMIT 3;")
cursor.fetchall()

[(1,
  False,
  3,
  'Braund, Mr. Owen Harris',
  'male      ',
  22.0,
  1,
  0,
  'A/5 21171',
  7.25,
  None,
  'S  '),
 (2,
  True,
  1,
  'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
  'female    ',
  38.0,
  1,
  0,
  'PC 17599',
  71.2833,
  'C85                 ',
  'C  '),
 (3,
  True,
  3,
  'Heikkinen, Miss. Laina',
  'female    ',
  26.0,
  0,
  0,
  'STON/O2. 3101282',
  7.925,
  None,
  'S  ')]

In [42]:
# Update passenger name for PassengerId 1 with your friend's name
cursor.execute("""
    UPDATE titanic_train
    SET name = 'Mark Zuck'
    WHERE PassengerId = 1;
    """)

In [43]:
# Remove all rows where passengers did not survive
cursor.execute("""
    DELETE FROM titanic_train
    WHERE survived = false;
""")

# Remove titanic_train table
cursor.execute("DROP TABLE IF EXISTS titanic_train")

### Clean-up

Let's remove the database we created solely for this notebook. You may keep it but we are not going to use it for the remainder of this course.

In [45]:
# Drop the "practice" database and return existing DBs
cursor.close()
conn.close()
conn = psycopg2.connect(
    user="postgres",
    password="asd",
    host="localhost",
    database="postgres"
)
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("DROP DATABASE IF EXISTS psycopg")
cursor.execute("SELECT datname FROM pg_database;")
display(cursor.fetchall())
cursor.close()
conn.close()

[('postgres',), ('suppliers',), ('template1',), ('template0',)]

## <a name="Summary">Summary 
After this tutorial, you should have been able to:

1. Create tables and rows in `PostgreSQL`,
2. Read results of tables and rows in `PostgreSQL`,
3. Update tables and rows in `PostgreSQL`, and
4. Delete tables and rows in `PostgreSQL`.

Congratulations, that concludes this lesson.

## Contributors
**Author**<br>
[Lee Kian Yang](https://github.com/KianYang-Lee)


## <a name="references">References</a>
- <a name="1">[1]</a> [psycopg Official Website](https://www.psycopg.org/)
- [PostgreSQL Python: Create Tables](https://www.postgresqltutorial.com/postgresql-python/create-tables/)
- [Psycopy 2.9.1 Documentation: Query Parameters](https://www.psycopg.org/docs/usage.html#query-parameters)
- [Passing Table Name as a Parameter in psycopg2](https://stackoverflow.com/questions/13793399/passing-table-name-as-a-parameter-in-psycopg2)